# Introduction

## 패스트캠퍼스 바이트디그리 2기 최종 프로젝트 과제 소개
### 과제:
1. SQL: SQL을 이용한 데이터 전처리 기법 습득
2. Python: EDA 과정에서의 결과 도출, 시각화 기법 습득



### 분석 목적:
패스트캠퍼스의 데이터를 활용하여 EDA 해보고 추세를 분석해보고
```
패스트캠퍼스의 사업팀에서 사용할 수 있는 지표와 대시보드를 기획(매출, 강의 방면)
```

### 사용 Tool:
- BigQuery: 데이터 import, 전처리
- Python: 분석


주졔:
```
사업 현황과 실적 개선을 위한 강의 기획에 관하여   
```
문졔:
```
연말연초외 실적 부진   
```
<img width="832" alt="월별 매출 추이" src="https://github.com/jayz0417/data-analysis-portfolio/assets/80396016/ce42aa01-ce3c-44af-84ac-00cae8266a84">


해결:
```
월 매출, 유저 행태, 인기 강의를 분석한다
```
1. 강의가 부족한 것은 아닐까?
2. 수강생이 부족한 것은 아닐까?
3. 객단가가 낮아서 그런 것은 아닐까?
4. 주문완료하지 않은 고객이 많은 건 아닐까?
5. 강의 현황
6. 신규 가입자 현황

결과:
```
1. 연말연초와 여름방학 시즌을 제외한 시기에 매출이 급감한다
2. 신규 가입자도 연밀연초, 여름방학 시즌에 증가한다
3. 인기 있는 강의는 데이터와 개발 관련 강의이다
4. 유저의 행태
- 주중 일과시간에 주로 신규 유저의 유입 많음
- 이탈 고객의 마지막 접속일은 분포상 50일이내가 많이 분포됨
- 신규 가입자는 연말연초, 여름방학 시즌 급증
- 코로나 팬데믹 전/후 변화(완화이후 급감)
```
결론:
```
1. 인기 강의와 연계할 수 있는 데이터/개발 관련 강의를 개발한다
2. 주중 일과시간에 신규 유저를 위한 프로모션을 기획한다
```

# 1. Data Extract - SQL


- 5개의 테이블(order, customer, course, refund, user)을 하나의 테이블로 병합

- (Order) 각 강의별 가장 첫 번째로 신청한 유저와 가장 마지막에 신청한 유저와 그 때의 날짜 구하기

- Order 테이블의 list_price를 월별로 합계한 후, 전달 대비 얼마나 증가하였는지, %지를 구하기

In [ ]:
#### 라이브러리 임포트
from google.colab import drive
import pandas as pd
import numpy as np
from plotnine import *
import seaborn as sns
import plotly.express as px

import gspread
from google.auth import default
creds, _ = default()
from gspread_dataframe import get_as_dataframe, set_with_dataframe

from google.cloud import bigquery
from oauth2client.client import GoogleCredentials

gc = gspread.authorize(creds)

In [ ]:
# 데이터프레임 숫자 변환(지수 표기법 -> 소수점)
pd.options.display.float_format = '{:0.2f}'.format

In [ ]:
query = """
  SELECT *
  FROM Fastcampus.order
  """

df_order = pd.io.gbq.read_gbq(
    query=query,
    project_id="jayz-316305"
)

df_order.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 284001 entries, 0 to 284000
Data columns (total 11 columns):
 #   Column          Non-Null Count   Dtype              
---  ------          --------------   -----              
 0   id              284001 non-null  Int64              
 1   type            284001 non-null  object             
 2   state           284001 non-null  object             
 3   name            284001 non-null  object             
 4   created_at      284001 non-null  datetime64[us, UTC]
 5   updated_at      284001 non-null  datetime64[us, UTC]
 6   customer_id     284001 non-null  Int64              
 7   list_price      284001 non-null  Int64              
 8   sale_price      284001 non-null  Int64              
 9   discount_price  284001 non-null  Int64              
 10  tax_free_price  284001 non-null  Int64              
dtypes: Int64(6), datetime64[us, UTC](2), object(3)
memory usage: 25.5+ MB


| Column          | Dtype                    | Description  |
|-----------------|--------------------------|--------------|
| id              | Int64                    | 주문id       |
| type            | object                   | 주문 타입 / unique = order |
| state           | object                   | 주문상태 / unique= ['COMPLETED', 'PENDING', 'CANCELLED'] |
| name            | object                   | 주문상품 명   |
| created_at      | datetime64[us, UTC]      | 생성시간      |
| updated_at      | datetime64[us, UTC]      | 업데이트시간  |
| customer_id     | Int64                    | 구매자id     |
| list_price      | Int64                    | 정상가       |
| sale_price      | Int64                    | 할인가       |
| discount_price  | Int64                    | 할인액       |
| tax_free_price  | Int64                    | 면세액       |


In [ ]:
query = """
  SELECT *
  FROM Fastcampus.course
  """

df_course = pd.io.gbq.read_gbq(
    query=query,
    project_id="jayz-316305"
)

df_course.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 722 entries, 0 to 721
Data columns (total 10 columns):
 #   Column             Non-Null Count  Dtype              
---  ------             --------------  -----              
 0   id                 722 non-null    Int64              
 1   type               722 non-null    object             
 2   state              722 non-null    object             
 3   created_at         722 non-null    datetime64[us, UTC]
 4   updated_at         722 non-null    datetime64[us, UTC]
 5   title              722 non-null    object             
 6   description        716 non-null    object             
 7   close_at           722 non-null    object             
 8   total_class_hours  722 non-null    object             
 9   keywords           715 non-null    object             
dtypes: Int64(1), datetime64[us, UTC](2), object(7)
memory usage: 57.2+ KB


| Column             | Dtype                  | Description   |
|--------------------|------------------------|---------------|
| id                 | Int64                  | 코스id        |
| type               | object                 | 코스타입 / unique=['COURSE', 'BOOK'] |
| state              | object                 | 코스상태 / unique=['ENDED', 'HIDDEN', 'ONGOING', 'PENDING'] |
| created_at         | datetime64[us, UTC]    | 생성시간       |
| updated_at         | datetime64[us, UTC]    | 업데이트시간   |
| title              | object                 | 타이틀        |
| description        | object                 | 설명          |
| close_at           | object                 | 마감시간       |
| total_class_hours  | object                 | 총 강의 시간   |
| keywords           | object                 | 키워드        |


In [ ]:
query = """
  SELECT *
  FROM Fastcampus.customer
  """

df_customer = pd.io.gbq.read_gbq(
    query=query,
    project_id="jayz-316305"
)

df_customer.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 123013 entries, 0 to 123012
Data columns (total 9 columns):
 #   Column      Non-Null Count   Dtype 
---  ------      --------------   ----- 
 0   id          123013 non-null  Int64 
 1   type        123013 non-null  object
 2   state       123013 non-null  object
 3   created_at  123013 non-null  object
 4   updated_at  123013 non-null  object
 5   user_id     123013 non-null  Int64 
 6   name        123013 non-null  object
 7   phone       123013 non-null  Int64 
 8   email       123013 non-null  object
dtypes: Int64(3), object(6)
memory usage: 8.8+ MB


| Column      | Dtype        | Description   |
|-------------|--------------|---------------|
| id          | Int64        | 구매자id      |
| type        | object       | 구매자타입 / unique=['USER'] |
| state       | object       | 구매자 상태 / unique=['NORMAL', 'DELETED', 'WITHDRAWN', 'HIBERNATED'] |
| created_at  | object       | 생성시간       |
| updated_at  | object       | 업데이트시간   |
| user_id     | Int64        | 유저id        |
| name        | object       | 구매자 이름    |
| phone       | Int64        | 구매자 연락처  |
| email       | object       | 구매자 이메일  |


In [ ]:
query = """
  SELECT *
  FROM Fastcampus.refund
  """

df_refund = pd.io.gbq.read_gbq(
    query=query,
    project_id="jayz-316305"
)

df_refund.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 12674 entries, 0 to 12673
Data columns (total 9 columns):
 #   Column           Non-Null Count  Dtype              
---  ------           --------------  -----              
 0   id               12674 non-null  Int64              
 1   type             12674 non-null  object             
 2   state            12674 non-null  object             
 3   created_at       12674 non-null  datetime64[us, UTC]
 4   updated_at       12674 non-null  datetime64[us, UTC]
 5   user_id          12674 non-null  object             
 6   course_id        12674 non-null  object             
 7   amount           12674 non-null  Int64              
 8   tax_free_amount  12674 non-null  Int64              
dtypes: Int64(3), datetime64[us, UTC](2), object(4)
memory usage: 928.4+ KB


| Column          | Dtype                  | Description   |
|-----------------|------------------------|---------------|
| id              | Int64                  | 환불id        |
| type            | object                 | 환불타입 / unique=['ADMIN_AUTO_REFUND'] |
| state           | object                 | 환불상태 / unique=['COMPLETED'] |
| created_at      | datetime64[us, UTC]    | 생성시간       |
| updated_at      | datetime64[us, UTC]    | 업데이트시간   |
| user_id         | object                 | 환불유저 id    |
| course_id       | object                 | 환불코스id     |
| amount          | Int64                  | 환불액         |
| tax_free_amount | Int64                  | 면세액         |


In [ ]:
query = """
  SELECT *
  FROM Fastcampus.user
  """

df_user = pd.io.gbq.read_gbq(
    query=query,
    project_id="jayz-316305"
)

df_user.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 122989 entries, 0 to 122988
Data columns (total 7 columns):
 #   Column         Non-Null Count   Dtype              
---  ------         --------------   -----              
 0   id             122989 non-null  Int64              
 1   type           122989 non-null  object             
 2   state          122989 non-null  object             
 3   created_at     122989 non-null  datetime64[us, UTC]
 4   updated_at     122989 non-null  datetime64[us, UTC]
 5   last_login_at  122989 non-null  datetime64[us, UTC]
 6   username       122989 non-null  object             
dtypes: Int64(1), datetime64[us, UTC](3), object(3)
memory usage: 6.7+ MB


| Column        | Dtype                    | Description   |
|---------------|--------------------------|---------------|
| id            | Int64                    | 유저id        |
| type          | object                   | 유저타입 / unique=['USER'] |
| state         | object                   | 유저상태 / unique=['NORMAL', 'ZOMBIE', 'DELETED', 'WITHDRAWN', 'HIBERNATED'] |
| created_at    | datetime64[us, UTC]      | 생성시간       |
| updated_at    | datetime64[us, UTC]      | 업데이트시간   |
| last_login_at | datetime64[us, UTC]      | 마지막 로그인 시간 |
| username      | object                   | 유저이름       |


#### Q1-1 JOIN

- Q1-1. 5개의 테이블(order, customer, course, refund, user)을 하나의 테이블로 조인하는 쿼리를 작성해주세요.

  - order 테이블과 customer 테이블은 customer_id와 id 필드를 기준으로 Inner Join
  - customer 테이블과 user 테이블은 user_id와 id 필드를 기준으로 Inner Join
  - user 테이블과 refund 테이블은 id와 user_id 필드를 기준으로 Left JOIN
    -  refund.user_id의 타입이 varchar이므로, 이를 정수형으로 변환
  - refund 테이블과 course 테이블은 course_id와 id 필드를 기준으로 Left Join
    - refund.course_id의 타입을 정수형으로 qusghks
  - 왼쪽 조인(LEFT JOIN)을 사용한 이유는 refund와 course 정보가 모든 주문에 대해 존재하지 않을 수 있기 때문

In [ ]:
query = """
  SELECT
    o.id AS order_id,
    o.type AS order_type,
    o.state AS order_state,
    o.name AS order_name,
    o.created_at AS order_created_at,
    o.updated_at AS order_updated_at,
    o.list_price,
    o.sale_price,
    o.discount_price,
    o.tax_free_price,
    c.title AS course_title,
    c.description AS course_description,
    cust.name AS customer_name,
    cust.phone AS customer_phone,
    cust.email AS customer_email,
    u.username AS user_username,
    u.created_at as user_created_at,
    u.state as user_state,
    u.last_login_at AS user_last_login_at,
    r.amount AS refund_amount,
    r.tax_free_amount AS refund_tax_free_amount
  FROM Fastcampus.order o
  INNER JOIN Fastcampus.customer cust --order 테이블과 customer 테이블은 customer_id와 id 필드를 기준으로 내부 조인
  ON o.customer_id = cust.id
  INNER JOIN Fastcampus.user u -- customer 테이블과 user 테이블은 user_id와 id 필드를 기준으로 내부 조인
  ON cust.user_id = u.id
  LEFT JOIN Fastcampus.refund r -- user 테이블과 refund 테이블은 id와 user_id 필드를 기준으로 왼쪽 조인
  ON u.id = SAFE_CAST(r.user_id AS INTEGER)
  LEFT JOIN Fastcampus.course c -- refund 테이블과 course 테이블은 course_id와 id 필드를 기준으로 왼쪽 조인
  ON SAFE_CAST(r.course_id AS INTEGER) = c.id
  """

df_join = pd.io.gbq.read_gbq(
    query=query,
    project_id="jayz-316305"
)

df_join.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 324418 entries, 0 to 324417
Data columns (total 21 columns):
 #   Column                  Non-Null Count   Dtype              
---  ------                  --------------   -----              
 0   order_id                324418 non-null  Int64              
 1   order_type              324418 non-null  object             
 2   order_state             324418 non-null  object             
 3   order_name              324418 non-null  object             
 4   order_created_at        324418 non-null  datetime64[us, UTC]
 5   order_updated_at        324418 non-null  datetime64[us, UTC]
 6   list_price              324418 non-null  Int64              
 7   sale_price              324418 non-null  Int64              
 8   discount_price          324418 non-null  Int64              
 9   tax_free_price          324418 non-null  Int64              
 10  course_title            78751 non-null   object             
 11  course_description      78

In [ ]:
df_join.head(10)

,order_id,order_type,order_state,order_name,order_created_at,order_updated_at,list_price,sale_price,discount_price,tax_free_price,...,course_description,customer_name,customer_phone,customer_email,user_username,user_created_at,user_state,user_last_login_at,refund_amount,refund_tax_free_amount
0,992031,ORDER,COMPLETED,Financial,2022-04-08 06:02:01+00:00,2022-04-08 06:02:02+00:00,30000,0,30000,0,...,None,USER_825158,1012345678,USER_825158@notexistemail.com,nhs1,2022-03-10 02:49:24+00:00,NORMAL,2023-12-04 01:41:39+00:00,<NA>,<NA>
1,987660,ORDER,COMPLETED,Financial,2022-04-06 02:25:26+00:00,2022-04-06 02:25:27+00:00,30000,0,30000,0,...,None,USER_742524,1012345678,USER_742524@notexistemail.com,mini,2021-12-20 02:01:52+00:00,NORMAL,2023-12-31 07:13:41+00:00,<NA>,<NA>
2,987576,ORDER,COMPLETED,Financial,2022-04-06 01:35:07+00:00,2022-04-06 01:35:08+00:00,30000,0,30000,0,...,None,USER_825091,1012345678,USER_825091@notexistemail.com,fina,2022-03-10 01:47:55+00:00,NORMAL,2023-09-07 00:37:46+00:00,<NA>,<NA>
3,869189,ORDER,COMPLETED,Financial,2022-02-07 04:16:50+00:00,2022-02-07 04:16:51+00:00,30000,0,30000,0,...,None,USER_772705,1012345678,USER_772705@notexistemail.com,choi,2022-01-14 01:23:02+00:00,NORMAL,2024-01-22 05:19:40+00:00,<NA>,<NA>
4,868984,ORDER,COMPLETED,Financial,2022-02-07 03:18:55+00:00,2022-02-07 03:18:56+00:00,30000,0,30000,0,...,None,USER_71928,1012345678,USER_71928@notexistemail.com,newo,2020-02-06 10:03:36+00:00,NORMAL,2023-12-12 00:07:26+00:00,<NA>,<NA>
5,868646,ORDER,COMPLETED,Financial,2022-02-07 01:55:35+00:00,2022-02-07 01:55:36+00:00,30000,0,30000,0,...,None,USER_774724,1012345678,USER_774724@notexistemail.com,leeh,2022-01-16 00:57:31+00:00,NORMAL,2023-08-23 06:00:06+00:00,<NA>,<NA>
6,868550,ORDER,COMPLETED,Financial,2022-02-07 01:40:52+00:00,2022-02-07 01:40:53+00:00,30000,0,30000,0,...,7명의 현업 전문가,USER_757150,1012345678,USER_757150@notexistemail.com,daro,2022-01-01 05:50:38+00:00,NORMAL,2023-02-24 00:37:23+00:00,-30000,-30000
7,811137,ORDER,COMPLETED,Financial,2022-01-13 07:31:12+00:00,2022-01-13 07:31:13+00:00,30000,0,30000,0,...,None,USER_139631,1012345678,USER_139631@notexistemail.com,oys5,2020-07-30 16:05:40+00:00,NORMAL,2024-01-12 09:09:39+00:00,<NA>,<NA>
8,796454,ORDER,COMPLETED,Financial,2022-01-07 14:57:14+00:00,2022-01-07 14:57:15+00:00,30000,0,30000,0,...,None,USER_616936,1012345678,USER_616936@notexistemail.com,imim,2021-07-31 02:31:11+00:00,NORMAL,2023-09-24 01:49:13+00:00,<NA>,<NA>
9,796420,ORDER,COMPLETED,Financial,2022-01-07 14:49:41+00:00,2022-01-07 14:49:42+00:00,30000,0,30000,0,...,None,USER_23734,1012345678,USER_23734@notexistemail.com,1204,2019-09-26 03:33:21+00:00,NORMAL,2023-07-09 10:34:38+00:00,<NA>,<NA>


#### Q1-2 PARTITION BY

- Q1-2. (Order) 각 강의별 가장 첫 번째로 신청한 유저와 가장 마지막에 신청한 유저와 그 때의 날짜를 구하시오. (First_Value, Last_value, partition by)
  - 가장 처음과 가장 마지막에 신청한 주문
    - PARTITION BY course_title: course_title별로 데이터를 그룹화하여 각 그룹 내에서 FIRST_VALUE와 LAST_VALUE 함수를 적용
    - ORDER BY order_created_at ASC: 각 파티션 내에서 order_created_at 기준으로 오름차순 정렬

  - RANGE BETWEEN UNBOUNDED PRECEDING AND UNBOUNDED FOLLOWING: LAST_VALUE 함수에 이 옵션을 사용하여 파티션 내의 모든 행을 참조
  - LAST_VALUE는 현재 행까지만 고려하기 때문에, 이 옵션 없이는 원하는 결과를 얻을 수 없음
  - DISTINCT: 각 코스별로 유일한 결과를 얻기 위해 쿼리 결과에서 중복을 제거

In [ ]:
query = """
  WITH base AS (
    SELECT
      o.id AS order_id,
      o.customer_id,
      o.type AS order_type,
      o.state AS order_state,
      o.name AS order_name,
      o.created_at AS order_created_at,
      o.updated_at AS order_updated_at,
      o.list_price,
      o.sale_price,
      o.discount_price,
      o.tax_free_price,
      c.title AS course_title,
      c.description AS course_description,
      cust.name AS customer_name,
      cust.phone AS customer_phone,
      cust.email AS customer_email,
      u.username AS user_username,
      u.last_login_at AS user_last_login_at,
      r.amount AS refund_amount,
      r.tax_free_amount AS refund_tax_free_amount
    FROM Fastcampus.order o
    INNER JOIN Fastcampus.customer cust --order 테이블과 customer 테이블은 customer_id와 id 필드를 기준으로 내부 조인
    ON o.customer_id = cust.id
    INNER JOIN Fastcampus.user u -- customer 테이블과 user 테이블은 user_id와 id 필드를 기준으로 내부 조인
    ON cust.user_id = u.id
    LEFT JOIN Fastcampus.refund r -- user 테이블과 refund 테이블은 id와 user_id 필드를 기준으로 왼쪽 조인
    ON u.id = SAFE_CAST(r.user_id AS INTEGER)
    LEFT JOIN Fastcampus.course c -- refund 테이블과 course 테이블은 course_id와 id 필드를 기준으로 왼쪽 조인
    ON SAFE_CAST(r.course_id AS INTEGER) = c.id
  )

  SELECT DISTINCT
    course_title,
    FIRST_VALUE(user_username) OVER(PARTITION BY course_title ORDER BY order_created_at ASC) AS first_user,
    FIRST_VALUE(customer_id) OVER(PARTITION BY course_title ORDER BY order_created_at ASC) AS first_cust_id,
    FIRST_VALUE(order_created_at) OVER(PARTITION BY course_title ORDER BY order_created_at ASC) AS first_order_date,
    LAST_VALUE(user_username) OVER(PARTITION BY course_title ORDER BY order_created_at ASC RANGE BETWEEN UNBOUNDED PRECEDING AND UNBOUNDED FOLLOWING) AS last_user,
    LAST_VALUE(customer_id) OVER(PARTITION BY course_title ORDER BY order_created_at ASC RANGE BETWEEN UNBOUNDED PRECEDING AND UNBOUNDED FOLLOWING) AS last_cust_id,
    LAST_VALUE(order_created_at) OVER(PARTITION BY course_title ORDER BY order_created_at ASC RANGE BETWEEN UNBOUNDED PRECEDING AND UNBOUNDED FOLLOWING) AS last_order_date
  FROM base
  WHERE course_title IS NOT NULL
  ORDER BY course_title ASC
  """

df_q2 = pd.io.gbq.read_gbq(
    query=query,
    project_id="jayz-316305"
)

df_q2.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 442 entries, 0 to 441
Data columns (total 7 columns):
 #   Column            Non-Null Count  Dtype              
---  ------            --------------  -----              
 0   course_title      442 non-null    object             
 1   first_user        442 non-null    object             
 2   first_cust_id     442 non-null    Int64              
 3   first_order_date  442 non-null    datetime64[us, UTC]
 4   last_user         442 non-null    object             
 5   last_cust_id      442 non-null    Int64              
 6   last_order_date   442 non-null    datetime64[us, UTC]
dtypes: Int64(2), datetime64[us, UTC](2), object(3)
memory usage: 25.2+ KB


In [ ]:
df_q2.head(10)

,course_title,first_user,first_cust_id,first_order_date,last_user,last_cust_id,last_order_date
0,15명의 전문 애-205707,wshi,441545,2022-01-01 14:36:24+00:00,ohip,36280,2022-12-31 14:53:11+00:00
1,The RED :-204652,sunw,762179,2022-01-05 10:45:02+00:00,song,1057259,2022-12-18 09:06:04+00:00
2,(B2B) 엔디에스-210539,mksh,444790,2022-03-07 06:34:56+00:00,mksh,444790,2022-03-07 06:35:40+00:00
3,(B2G)K-Dig-212428,pink,896337,2022-07-20 04:21:40+00:00,pink,896337,2022-07-20 15:43:42+00:00
4,100개의 클론디자-208383,didi,393962,2022-01-02 15:33:32+00:00,2cir,47434,2022-12-31 09:42:01+00:00
5,100억을 움직이는-203499,viru,757801,2022-01-02 02:59:40+00:00,oliv,1027066,2022-12-31 23:25:37+00:00
6,100억을 움직이는-208104,sand,6086,2022-01-03 04:53:15+00:00,eom0,1019444,2022-12-30 05:36:50+00:00
7,10개 프로젝트로 -211368,eent,32020,2022-01-03 02:09:41+00:00,loli,1069042,2022-12-31 12:14:37+00:00
8,10년차 패션 MD-211177,mhsh,138095,2022-01-06 09:06:45+00:00,mhsh,138095,2022-12-28 06:14:02+00:00
9,120일 만에 데이-211585,sell,748184,2022-06-06 10:10:30+00:00,sell,748184,2022-06-07 08:39:46+00:00


#### Q1-3 LEAD, LAG

- Q1-3. Order 테이블의 list_price를 월별로 합계한 후, 전달 대비 얼마나 증가하였는지, %지를 구하시오(lead, lag)

  - 월별 합계 계산: WITH 절을 사용하여 MonthlySums라는 임시 테이블을 생성하고, 여기서 각 월의 list_price 합계를 계산
  - 전월 대비 변화 계산: 다음 WITH 절인 MonthlyChanges에서는 LAG() 함수를 사용하여 각 월의 합계와 이전 달의 합계를 비교
    - change_amount는 전달 대비 증가액을 나타내고, change_percent는 증가 비율을 나타냄
  - 결과 선택 및 정렬: 마지막 SELECT 문에서는 계산된 월별 총액, 증가액, 및 증가 비율을 선택하고, 이를 연도와 월별로 정렬

In [ ]:
query = """
  WITH base AS (
    SELECT
      o.id AS order_id,
      o.customer_id,
      o.type AS order_type,
      o.state AS order_state,
      o.name AS order_name,
      o.created_at AS order_created_at,
      o.updated_at AS order_updated_at,
      o.list_price,
      o.sale_price,
      o.discount_price,
      o.tax_free_price,
      c.title AS course_title,
      c.description AS course_description,
      cust.name AS customer_name,
      cust.phone AS customer_phone,
      cust.email AS customer_email,
      u.username AS user_username,
      u.last_login_at AS user_last_login_at,
      r.amount AS refund_amount,
      r.tax_free_amount AS refund_tax_free_amount
    FROM Fastcampus.order o
    INNER JOIN Fastcampus.customer cust --order 테이블과 customer 테이블은 customer_id와 id 필드를 기준으로 내부 조인
    ON o.customer_id = cust.id
    INNER JOIN Fastcampus.user u -- customer 테이블과 user 테이블은 user_id와 id 필드를 기준으로 내부 조인
    ON cust.user_id = u.id
    LEFT JOIN Fastcampus.refund r -- user 테이블과 refund 테이블은 id와 user_id 필드를 기준으로 왼쪽 조인
    ON u.id = SAFE_CAST(r.user_id AS INTEGER)
    LEFT JOIN Fastcampus.course c -- refund 테이블과 course 테이블은 course_id와 id 필드를 기준으로 왼쪽 조인
    ON SAFE_CAST(r.course_id AS INTEGER) = c.id
  ),

  MonthlySums AS (
    SELECT
      EXTRACT(YEAR FROM order_created_at AT TIME ZONE 'Asia/Seoul') AS year,
      EXTRACT(MONTH FROM order_created_at) AS month,
      SUM(list_price) AS total_list_price
    FROM base
    GROUP BY year, month
  ),

  MonthlyChanges AS (
    SELECT
      year,
      month,
      total_list_price,
      LAG(total_list_price) OVER (ORDER BY year, month) AS prev_month_total,
      total_list_price - LAG(total_list_price) OVER (ORDER BY year, month) AS change_amount,
      ((total_list_price - LAG(total_list_price) OVER (ORDER BY year, month)) / LAG(total_list_price) OVER (ORDER BY year, month)) * 100 AS change_percent
    FROM MonthlySums
  )

  SELECT
    year,
    month,
    total_list_price,
    change_amount,
    change_percent
  FROM MonthlyChanges
  ORDER BY year, month
  """

df_q3 = pd.io.gbq.read_gbq(
    query=query,
    project_id="jayz-316305"
)

df_q3.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 13 entries, 0 to 12
Data columns (total 5 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   year              13 non-null     Int64  
 1   month             13 non-null     Int64  
 2   total_list_price  13 non-null     Int64  
 3   change_amount     12 non-null     Int64  
 4   change_percent    12 non-null     float64
dtypes: Int64(4), float64(1)
memory usage: 700.0 bytes


In [ ]:
df_q3.head(10)

,year,month,total_list_price,change_amount,change_percent
0,2022,1,1007690000,<NA>,NaN
1,2022,2,731240000,-276450000,-27.43
2,2022,3,673570000,-57670000,-7.89
3,2022,4,669350000,-4220000,-0.63
4,2022,5,570040000,-99310000,-14.84
5,2022,6,650800000,80760000,14.17
6,2022,7,794940000,144140000,22.15
7,2022,8,748980000,-45960000,-5.78
8,2022,9,707710000,-41270000,-5.51
9,2022,10,695380000,-12330000,-1.74


# 2. Data Analysis & Visualization
- **Analysis Too**l: Python, Excel, Tableau
- **Visualization Tool**: Tableau , Power BI, Excel, Python(matplotlib, seaborn, plotly)

### 2.1 Descriptive Analysis

In [ ]:
df_join.describe()

,order_id,list_price,sale_price,discount_price,tax_free_price,customer_phone,refund_amount,refund_tax_free_amount
count,324418.00,324418.00,324418.00,324418.00,324418.00,324418.00,86602.00,86602.00
mean,1189540.94,28323.18,23491.54,7372.62,22275.92,1012345678.00,-17689.20,-15612.57
std,247755.33,36745.65,37135.56,11665.40,35196.14,0.00,20749.34,14032.44
min,773390.00,10000.00,0.00,0.00,0.00,1012345678.00,-600000.00,-600000.00
25%,971966.75,20000.00,20000.00,0.00,20000.00,1012345678.00,-20000.00,-20000.00
50%,1191935.50,30000.00,20000.00,0.00,20000.00,1012345678.00,-10000.00,-10000.00
75%,1408755.75,30000.00,30000.00,10000.00,30000.00,1012345678.00,-10000.00,-10000.00
max,1600585.00,6690000.00,6690000.00,200000.00,6690000.00,1012345678.00,0.00,0.00


In [ ]:
df_join['order_created_at'].max()

Timestamp('2022-12-31 23:42:49+0000', tz='UTC')

In [ ]:
df_join['order_created_at'].min()

Timestamp('2022-01-01 00:08:19+0000', tz='UTC')

#### Q2-1, Q2-2

Q2-1 (Order) 2022년 1월 중 가장 매출이 높았던 3일과 낮았던 3일은 언제인가요? (status = COMPLETED 기준)

- 가장 매출이 높았던 3일:

| order_created_at | total_sales | order_count | customer_count | course_count |
|------------------|-------------|-------------|----------------|--------------|
| 2022-01-03       | 52070000    | 1862        | 1670           | 157          |
| 2022-01-07       | 40230000    | 1305        | 1136           | 145          |
| 2022-01-09       | 38410000    | 1195        | 1058           | 157   

- 가장 매출이 낮았던 3일:

| order_created_at | total_sales | order_count | customer_count | course_count |
|------------------|-------------|-------------|----------------|--------------|
| 2022-01-10       | 10280000    | 346         | 332            | 63           |
| 2022-01-24       | 9670000     | 330         | 305            | 31           |
| 2022-01-01       | 8080000     | 236         | 221            | 49           |


Q2-2 (Order) 그리고 그 때의 매출액(list_price)과, 주문 수, 신청자 수, 강의 수는 각각 얼마인가요?

<img width="820" alt="Top10 키워드" src="https://github.com/jayz0417/data-analysis-portfolio/assets/80396016/2b4613ba-7d43-4ee5-b872-905a855d771c">

In [ ]:
# 주문 상태가 'COMPLETED'이고 2022년 1월인 데이터 필터링
df_completed = df_join[(df_join['order_state'] == 'COMPLETED') &
                       (df_join['order_created_at'].dt.year == 2022) &
                       (df_join['order_created_at'].dt.month == 1)]

# 날짜별로 그룹화하여 매출액 합계, 주문 수, 신청자 수, 강의 수 계산
sales_summary = df_completed.groupby(df_completed['order_created_at'].dt.date).agg(
    total_sales=('list_price', 'sum'),
    order_count=('order_id', 'nunique'),
    customer_count=('customer_name', 'nunique'),
    course_count=('course_title', 'nunique')
).sort_values(by='total_sales', ascending=False)

# 가장 매출이 높았던 3일과 낮았던 3일
top_3_sales_days = sales_summary.head(3)
bottom_3_sales_days = sales_summary.tail(3)

print("가장 매출이 높았던 3일:\n", top_3_sales_days)
print("\n가장 매출이 낮았던 3일:\n", bottom_3_sales_days)

가장 매출이 높았던 3일:
                   total_sales  order_count  customer_count  course_count
order_created_at                                                        
2022-01-03           52070000         1862            1670           157
2022-01-07           40230000         1305            1136           145
2022-01-09           38410000         1195            1058           157

가장 매출이 낮았던 3일:
                   total_sales  order_count  customer_count  course_count
order_created_at                                                        
2022-01-10           10280000          346             332            63
2022-01-24            9670000          330             305            31
2022-01-01            8080000          236             221            49


#### Q2-3

Q2-3 (Order) 월별 각 강의 별 할인율은 어떻게 될까요?

In [ ]:
# 할인율 계산 후 새로운 컬럼에 저장
df_join['discount_rate'] = ((df_join['list_price'] - df_join['sale_price']) / df_join['list_price']) * 100

# 월별, 강의별로 할인율 평균 계산
monthly_discount_rate = df_join.groupby([df_join['order_created_at'].dt.to_period('M'), 'course_title'])['discount_rate'].mean()

print(monthly_discount_rate)

order_created_at  course_title     
2022-01            15명의 전문 애-205707    4.55
                   The RED :-204652    3.82
                  100개의 클론디자-208383    7.69
                  100억을 움직이는-203499    4.88
                  100억을 움직이는-208104   13.65
                                       ... 
2022-12           해외주식명저로 배우-208686   12.82
                  혁펜하임의 AI D-213780   37.44
                  화사한 색감과 빛의-211807   72.84
                  확실하게 끝내는 포-204207   31.25
                  히어로부터 크리쳐까-207779   33.33
Name: discount_rate, Length: 4433, dtype: Float64


<ipython-input-20-f3c83b2935db>:5: UserWarning: Converting to PeriodArray/Index representation will drop timezone information.


#### Q2-4

Q2-4 (Course) 강의 키워드에서 가장 많이 언급된 단어는 무엇인가요?

In [ ]:
from collections import Counter

# 코스 키워드에서 가장 많이 언급된 단어 계산
keywords = df_join['course_description'].dropna().str.split().explode().values
most_common_word = Counter(keywords).most_common(1)

print("가장 많이 언급된 단어:", most_common_word)

가장 많이 언급된 단어: [('데이터', 2689)]


In [ ]:
# 강의 설명에서 모든 단어 추출
keywords = df_join['course_description'].dropna().str.split().explode().values

# 단어별 등장 빈도 계산
word_counts = Counter(keywords)

# 가장 많이 언급된 상위 10개 단어 추출
most_common_words = word_counts.most_common(10)

# 결과 출력
print("가장 많이 언급된 키워드 Top 10:")
for word, count in most_common_words:
    print(f"{word}: {count}")

가장 많이 언급된 키워드 Top 10:
데이터: 2689
국내: 2434
네오아카데미: 1749
요구사항: 1521
분석,: 1521
도: 1521
구: 1368
아트디: 1362
프론트엔드: 1304
필요한: 1299


트리맵으로 바꿔보기

In [ ]:
# 1단계: 매출이 높은 상위 10개의 order_name 선정
top_10_sales_orders = df_join.groupby('order_name')['list_price'].sum().nlargest(10).index.tolist()

# 2단계: 선정된 order_name에 대한 키워드 추출
top_10_sales_keywords = df_join[df_join['order_name'].isin(top_10_sales_orders)]['course_description'].dropna().str.split().explode().values

# 3단계: 키워드 빈도 분석
keyword_counts = Counter(top_10_sales_keywords)
top_10_keywords = keyword_counts.most_common(10)

# 4단계: 결과 시각화
keywords, counts = zip(*top_10_keywords)  # 키워드와 빈도수를 분리하여 저장
fig = px.bar(x=keywords, y=counts, labels={'x': 'Keyword', 'y': 'Frequency'}, title='Top 10 Keywords in Top 10 강의네임')
fig.show()

In [ ]:
# 키워드와 빈도수를 데이터프레임으로 변환
df_top_keywords = pd.DataFrame({'Keyword': keywords, 'Frequency': counts})

# 트리맵 시각화
fig = px.treemap(df_top_keywords,
                 path=['Keyword'],
                 values='Frequency',
                 title='Top 10 Keywords in Top 10 Sales Order Names')

fig.data[0].textinfo = 'label+text+value'
fig.show()

#### Q2-5

Q2-5 (Refund) 환불이 가장 많은 강의는 어떤 강의였나요?

In [ ]:
most_refunded_course = df_join.groupby('course_title')['refund_amount'].sum().idxmax()
print("환불이 가장 많은 강의:", most_refunded_course)

환불이 가장 많은 강의: 10년차 패션 MD-211177


#### Q2-6

Q2-6 (Customer) 강의를 가장 많이 신청한 고객과 그 금액은 얼마인가요?

In [ ]:
top_customer = df_join.groupby('customer_name')['list_price'].sum().idxmax()
top_customer_amount = df_join.groupby('customer_name')['list_price'].sum().max()
print(f"가장 많이 신청한 고객: {top_customer}, 금액: {top_customer_amount}")


가장 많이 신청한 고객: USER_138095, 금액: 94600000


In [ ]:
# 고객별 신청 강의 수 계산
course_count_per_customer = df_join.groupby('customer_name').size().reset_index(name='course_count')

# 고객별 신청한 강의의 총 금액 계산
total_price_per_customer = df_join.groupby('customer_name')['list_price'].sum().reset_index(name='total_price')

# 두 데이터프레임 합치기
customer_data = pd.merge(course_count_per_customer, total_price_per_customer, on='customer_name')

# 스캐터 플롯 생성
fig = px.scatter(customer_data, x='course_count', y='total_price',
                 title="고객별 신청 강의 수와 금액 분포",
                 labels={'course_count': '신청 강의 수', 'total_price': '총 금액'},
                 hover_data=['customer_name'])

fig.show()

아웃라이어의 특징이 개인 고객이라고 하기에는 신청 강의 수와 총 금액이 너무 크기 때문에    

기업 고객(B2B)가 아닐까


#### Q2-7

Q2-7 (Order) 주말과 주중의 평균 강의 신청율은 어떻게 다를까요?

In [ ]:
# 주말과 주중을 구분하기 위한 'day_of_week' 컬럼 추가
df_join['day_of_week'] = df_join['order_created_at'].dt.dayofweek

# 주말과 주중 데이터를 분리
## day_of_week 값이 5(토요일) 또는 6(일요일)인 경우 주말, 그외 주중
weekend_data = df_join[df_join['day_of_week'] > 4]
weekday_data = df_join[df_join['day_of_week'] <= 4]

# 주말 강의 신청율 계산
weekend_completed_orders = weekend_data[weekend_data['order_state'] == 'COMPLETED']['order_id'].count()
weekend_total_orders = weekend_data['order_id'].count()
weekend_enrollment_rate = (weekend_completed_orders / weekend_total_orders) if weekend_total_orders > 0 else 0

# 주중 강의 신청율 계산
weekday_completed_orders = weekday_data[weekday_data['order_state'] == 'COMPLETED']['order_id'].count()
weekday_total_orders = weekday_data['order_id'].count()
weekday_enrollment_rate = (weekday_completed_orders / weekday_total_orders) if weekday_total_orders > 0 else 0

print(f"주말 강의 신청율: {weekend_enrollment_rate * 100:.2f}%")
print(f"주중 강의 신청율: {weekday_enrollment_rate * 100:.2f}%")

주말 강의 신청율: 61.32%
주중 강의 신청율: 62.91%


#### Q2-8

Q2-8 (User) 주로 몇시에 회원 가입(created_at)이 활발했나요? 월별로 차이가 있는지 보여주고, 그 근거를 추론해 보세요

> 분석   
- 시간대: 대체로 심야, 오전 시간대
```
일과 시간 전후 활동 증가: 심야, 오전
새벽 시간의 활동: 0시나 1시 같은 새벽 시간에 활발
```

>추론1
- 사용자 행동 패턴 분석
```
패스트캠퍼스를 이용하는 유저의 행동 패턴 추세가 이러한 것은 아닐까?
```

>검증1
- 주중 일과 시간동안 많이 가입함


>추론2
- 팬데믹 시기 동안 활동 패턴 변화
```
팬데믹 (2020년 2월부터 2022년)의 영향은 아닐까
```

> 검증2
- 코로나로 가입자가 늘어났고 코로나가 완화되며 가입이 감소
  - 재택근무, 코로나로 인한 구직난 등이 영향을 주었을 것으로 예상

In [ ]:
df_join['signup_hour'] = df_join['user_created_at'].dt.hour
monthly_signup_activity = df_join.groupby([df_join['user_created_at'].dt.to_period('M'), 'signup_hour']).size().unstack(fill_value=0)

print(monthly_signup_activity.idxmax(axis=1))

user_created_at
2019-07    23
2019-08     0
2019-09     1
2019-10    12
2019-11     9
2019-12    15
2020-01     6
2020-02     9
2020-03    14
2020-04    14
2020-05     8
2020-06    14
2020-07     4
2020-08    11
2020-09     0
2020-10     7
2020-11     9
2020-12     3
2021-01     3
2021-02    10
2021-03     7
2021-04    17
2021-05     9
2021-06    14
2021-07    15
2021-08     1
2021-09     4
2021-10    13
2021-11     2
2021-12    13
2022-01     9
2022-02     5
2022-03     5
2022-04    14
2022-05     7
2022-06    14
2022-07     5
2022-08    14
2022-09    13
2022-10    14
2022-11     5
2022-12     6
Freq: M, dtype: int32


<ipython-input-28-d53e65fb7fc9>:2: UserWarning:

Converting to PeriodArray/Index representation will drop timezone information.



In [ ]:
# 히트맵 생성
fig = px.imshow(monthly_signup_activity,
                labels=dict(x="Hour of Day", y="Month", color="Signup Count"),
                x=[str(hour) for hour in range(24)], # X축에 0부터 23시까지의 시간대를 나타냅니다.
                y=monthly_signup_activity.index.strftime('%Y-%m'), # Y축에 월을 나타냅니다.
                aspect="auto",
                color_continuous_scale='Viridis') # 컬러 스케일 설정

# 타이틀 설정
fig.update_layout(title_text='Monthly Signup Activity by Hour')

# X축 설정
fig.update_xaxes(side="top") # X축을 위쪽으로 이동

fig.show()

In [ ]:
## 사용자 행동 패턴 분석

import plotly.graph_objects as go

# 시간대별 회원 가입 수 집계
hourly_signup = df_join.groupby(df_join['signup_hour']).size()

# 시간대별 회원 가입 활동 시각화
fig = go.Figure()
fig.add_trace(go.Bar(x=hourly_signup.index, y=hourly_signup.values,
                     marker_color='lightblue',
                     name='Hourly Signups'))

fig.update_layout(title='시간대별 가입활동',
                  xaxis_title='Hour of Day',
                  yaxis_title='Number of Signups',
                  barmode='group')
fig.show()

In [ ]:
## 사용자 행동 패턴 분석

# 주말과 주중에 따른 회원 가입 활동 분류
df_join['is_weekend'] = df_join['day_of_week'].apply(lambda x: 'Weekend' if x > 4 else 'Weekday')
weekend_weekday_signup = df_join.groupby('is_weekend')['order_id'].count()

# 주말과 주중의 회원 가입 활동 시각화
fig = px.pie(weekend_weekday_signup, values='order_id', names=weekend_weekday_signup.index, title='주중/주말의 가입 비중')
fig.show()

In [ ]:
## 팬데믹 시기 동안 활동 패턴 변화

# 팬데믹 기간별로 데이터 분리
initial_phase = df_join[(df_join['order_created_at'] >= '2020-02') & (df_join['order_created_at'] < '2021-07')]
peak_phase = df_join[(df_join['order_created_at'] >= '2021-07') & (df_join['order_created_at'] < '2022-09')]
easing_phase = df_join[(df_join['order_created_at'] >= '2022-09')]

# 각 기간별 시간대별 회원 가입 활동 집계
initial_hourly = initial_phase.groupby(initial_phase['signup_hour']).size()
peak_hourly = peak_phase.groupby(peak_phase['signup_hour']).size()
easing_hourly = easing_phase.groupby(easing_phase['signup_hour']).size()

# 시각화
fig = go.Figure()

fig.add_trace(go.Bar(x=initial_hourly.index, y=initial_hourly.values, name='Initial Phase', marker_color='salmon'))
fig.add_trace(go.Bar(x=peak_hourly.index, y=peak_hourly.values, name='Peak Phase', marker_color='lightblue'))
fig.add_trace(go.Bar(x=easing_hourly.index, y=easing_hourly.values, name='Easing Phase', marker_color='yellowgreen'))

fig.update_layout(
    title='팬데믹 기간 시간대별 가입 활동의 변화',
    xaxis_title='Hour of Day',
    yaxis_title='Number of Signups',
    barmode='group'
)

fig.show()

#### Q2-9

Q2-9 (Order) 강의 가격(list_price)을 5만원 단위의 도수분포표로 나타내 주세요.

In [ ]:
price_bins = pd.cut(df_join['list_price'], bins=range(0, int(df_join['list_price'].max() + 50000), 50000))
frequency_distribution = df_join.groupby(price_bins)['order_id'].count()

print(frequency_distribution)

list_price
(0, 50000]            315251
(50000, 100000]         7848
(100000, 150000]         752
(150000, 200000]          36
(200000, 250000]         177
                       ...  
(6450000, 6500000]         0
(6500000, 6550000]         0
(6550000, 6600000]         0
(6600000, 6650000]         0
(6650000, 6700000]         4
Name: order_id, Length: 134, dtype: int64


In [ ]:
frequency_distribution

,list_price,order_id,sort_key
0,"(0, 50000]",315251,0
1,"(50000, 100000]",7848,50000
2,"(100000, 150000]",752,100000
3,"(150000, 200000]",36,150000
4,"(200000, 250000]",177,200000
...,...,...,...
129,"(6450000, 6500000]",0,6450000
130,"(6500000, 6550000]",0,6500000
131,"(6550000, 6600000]",0,6550000
132,"(6600000, 6650000]",0,6600000


In [ ]:
# 가격 범위 생성 및 도수분포 계산
price_bins = pd.cut(df_join['list_price'], bins=range(0, int(df_join['list_price'].max() + 50000), 50000))
frequency_distribution = df_join.groupby(price_bins)['order_id'].count().reset_index()

# 'list_price' 열의 Interval 객체를 문자열로 변환
frequency_distribution['list_price'] = frequency_distribution['list_price'].astype(str)

# 수정된 도수분포 시각화
fig = px.bar(frequency_distribution, x='list_price', y='order_id',
             labels={'list_price': 'List Price Range', 'order_id': 'Frequency'},
             title='Frequency Distribution of List Price')

# 가격 범위가 문자열이므로, 범위의 시작값을 기준으로 정렬하기 위해 추가 처리가 필요할 수 있습니다.
# 'list_price'를 기준으로 오름차순 정렬
frequency_distribution['sort_key'] = frequency_distribution['list_price'].apply(lambda x: int(x.split(',')[0][1:]))
frequency_distribution = frequency_distribution.sort_values('sort_key')

# 정렬된 데이터로 다시 그래프를 그립니다.
fig = px.bar(frequency_distribution, x='list_price', y='order_id',
             labels={'list_price': 'List Price Range', 'order_id': 'Frequency'},
             title='Frequency Distribution of List Price')

fig.show()

#### Q2-10

Q2-10 (Order) 신청이 완료된 강의(complated)와 진행중인 강의(pending), 그리고 취소된 강의(cancelled) 간의 금액(list_price)과 유저의 비중을 월별로 각각 구하시오

- 월별 주문 상태별 매출 합계

| order_created_at | CANCELLED  | COMPLETED  | PENDING    |
|------------------|------------|------------|------------|
| 2022-01          | 242,870,000| 594,730,000| 170,090,000|
| 2022-02          | 161,050,000| 458,490,000| 111,700,000|
| 2022-03          | 164,980,000| 403,770,000| 104,820,000|
| 2022-04          | 151,080,000| 397,140,000| 121,130,000|
| 2022-05          | 124,430,000| 350,240,000| 95,370,000 |
| 2022-06          | 151,130,000| 385,010,000| 114,660,000|
| 2022-07          | 190,060,000| 467,620,000| 137,260,000|
| 2022-08          | 160,740,000| 416,090,000| 172,150,000|
| 2022-09          | 167,930,000| 416,300,000| 123,480,000|
| 2022-10          | 148,730,000| 434,850,000| 111,800,000|
| 2022-11          | 164,460,000| 473,420,000| 114,990,000|
| 2022-12          | 203,170,000| 781,910,000| 200,900,000|


- 월별 주문 상태별 사용자 비율

| order_created_at | CANCELLED | COMPLETED | PENDING |
|------------------|-----------|-----------|---------|
| 2022-01          | 0.08      | 0.22      | 0.05    |
| 2022-02          | 0.06      | 0.18      | 0.04    |
| 2022-03          | 0.05      | 0.16      | 0.04    |
| 2022-04          | 0.05      | 0.15      | 0.03    |
| 2022-05          | 0.04      | 0.14      | 0.03    |
| 2022-06          | 0.05      | 0.16      | 0.04    |
| 2022-07          | 0.06      | 0.18      | 0.04    |
| 2022-08          | 0.06      | 0.18      | 0.04    |
| 2022-09          | 0.06      | 0.17      | 0.04    |
| 2022-10          | 0.05      | 0.17      | 0.04    |
| 2022-11          | 0.05      | 0.


In [ ]:
monthly_status_summary = df_join.groupby([df_join['order_created_at'].dt.to_period('M'), 'order_state'])['list_price'].sum()
user_proportion = df_join.groupby([df_join['order_created_at'].dt.to_period('M'), 'order_state'])['user_username'].nunique() / df_join['user_username'].nunique()

print(monthly_status_summary)
print(user_proportion)

<ipython-input-56-5346021189d3>:1: UserWarning:

Converting to PeriodArray/Index representation will drop timezone information.

<ipython-input-56-5346021189d3>:2: UserWarning:

Converting to PeriodArray/Index representation will drop timezone information.



order_created_at  order_state
2022-01           CANCELLED      242870000
                  COMPLETED      594730000
                  PENDING        170090000
2022-02           CANCELLED      161050000
                  COMPLETED      458490000
                  PENDING        111700000
2022-03           CANCELLED      164980000
                  COMPLETED      403770000
                  PENDING        104820000
2022-04           CANCELLED      151080000
                  COMPLETED      397140000
                  PENDING        121130000
2022-05           CANCELLED      124430000
                  COMPLETED      350240000
                  PENDING         95370000
2022-06           CANCELLED      151130000
                  COMPLETED      385010000
                  PENDING        114660000
2022-07           CANCELLED      190060000
                  COMPLETED      467620000
                  PENDING        137260000
2022-08           CANCELLED      160740000
                  COMPLE

In [ ]:
# user_proportion을 데이터프레임으로 변환 (컬럼 이름 추가)
df_user_proportion = user_proportion.reset_index()
df_user_proportion['order_created_at'] = df_user_proportion['order_created_at'].astype(str)
df_user_proportion.columns = ['order_created_at', 'order_state', 'user_proportion']  # 컬럼 이름 명시적으로 지정

# user_proportion을 피벗 테이블 형태로 변환 (수정된 컬럼 이름 사용)
pivot_user_proportion = df_user_proportion.pivot_table(index='order_created_at', columns='order_state', values='user_proportion')

print("Monthly Status Summary (List Price Sum by State):\n", pivot_monthly_status_summary, "\n")
print("User Proportion by Order State:\n", pivot_user_proportion)

Monthly Status Summary (List Price Sum by State):
 order_state         CANCELLED    COMPLETED      PENDING
order_created_at                                       
2022-01          242870000.00 594730000.00 170090000.00
2022-02          161050000.00 458490000.00 111700000.00
2022-03          164980000.00 403770000.00 104820000.00
2022-04          151080000.00 397140000.00 121130000.00
2022-05          124430000.00 350240000.00  95370000.00
2022-06          151130000.00 385010000.00 114660000.00
2022-07          190060000.00 467620000.00 137260000.00
2022-08          160740000.00 416090000.00 172150000.00
2022-09          167930000.00 416300000.00 123480000.00
2022-10          148730000.00 434850000.00 111800000.00
2022-11          164460000.00 473420000.00 114990000.00
2022-12          203170000.00 781910000.00 200900000.00 

User Proportion by Order State:
 order_state       CANCELLED  COMPLETED  PENDING
order_created_at                               
2022-01                0.08       

In [ ]:
## 금액 만원 단위 환산
pivot_monthly_status_summary_scaled = pivot_monthly_status_summary.apply(lambda x: x / 10000)

print("Monthly Status Summary (List Price Sum by State in 10,000 KRW Units):\n", pivot_monthly_status_summary_scaled, "\n")
print("User Proportion by Order State:\n", pivot_user_proportion)

Monthly Status Summary (List Price Sum by State in 10,000 KRW Units):
 order_state       CANCELLED  COMPLETED  PENDING
order_created_at                               
2022-01            24287.00   59473.00 17009.00
2022-02            16105.00   45849.00 11170.00
2022-03            16498.00   40377.00 10482.00
2022-04            15108.00   39714.00 12113.00
2022-05            12443.00   35024.00  9537.00
2022-06            15113.00   38501.00 11466.00
2022-07            19006.00   46762.00 13726.00
2022-08            16074.00   41609.00 17215.00
2022-09            16793.00   41630.00 12348.00
2022-10            14873.00   43485.00 11180.00
2022-11            16446.00   47342.00 11499.00
2022-12            20317.00   78191.00 20090.00 

User Proportion by Order State:
 order_state       CANCELLED  COMPLETED  PENDING
order_created_at                               
2022-01                0.08       0.22     0.05
2022-02                0.06       0.18     0.04
2022-03                0.05   

In [ ]:
from plotly.subplots import make_subplots

# 상태별 월별 금액 집계
monthly_status_summary = df_join.groupby([df_join['order_created_at'].dt.to_period('M'), 'order_state'])['list_price'].sum().unstack().fillna(0)

# 상태별 월별 유저 비중 집계
user_proportion = (df_join.groupby([df_join['order_created_at'].dt.to_period('M'), 'order_state'])['user_username'].nunique() / df_join.groupby(df_join['order_created_at'].dt.to_period('M'))['user_username'].nunique()).unstack().fillna(0)

# 서브플롯 생성
fig = make_subplots(rows=2, cols=1, shared_xaxes=True,
                    subplot_titles=('Monthly Total List Price by Order State', 'Monthly User Proportion by Order State'))

# 금액 시각화
for state in monthly_status_summary.columns:
    fig.add_trace(go.Bar(x=monthly_status_summary.index.astype(str), y=monthly_status_summary[state], name=state), row=1, col=1)

# 유저 비중 시각화
for state in user_proportion.columns:
    fig.add_trace(go.Bar(x=user_proportion.index.astype(str), y=user_proportion[state], name=state), row=2, col=1)

# 레이아웃 업데이트
fig.update_layout(height=600, showlegend=True, title_text="Monthly Summary by Order State")
fig.update_yaxes(title_text="Total List Price", row=1, col=1)
fig.update_yaxes(title_text="User Proportion", row=2, col=1)
fig.show()

<ipython-input-36-12e8060e10f3>:4: UserWarning:

Converting to PeriodArray/Index representation will drop timezone information.

<ipython-input-36-12e8060e10f3>:7: UserWarning:

Converting to PeriodArray/Index representation will drop timezone information.

<ipython-input-36-12e8060e10f3>:7: UserWarning:

Converting to PeriodArray/Index representation will drop timezone information.



#### Q2-11

Q2-11 (User) state이 HIBERNATED 된 Used들은 ID를 생성(created_at)하고 마지막 로그인(last_login_at)까지 평균적으로 몇일이 걸렸나요?

In [ ]:
df_hibernated = df_join[df_join['user_state'] == 'HIBERNATED']
df_hibernated['days_to_last_login'] = (df_hibernated['user_last_login_at'] - df_hibernated['user_created_at']).dt.days
average_days = df_hibernated['days_to_last_login'].mean()

print("Hibernated 상태의 유저들의 평균 일수:", average_days)

<ipython-input-37-361ec3bac749>:2: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy



Hibernated 상태의 유저들의 평균 일수: 180.39996530087038


In [ ]:
# 'HIBERNATED' 상태의 유저들의 마지막 로그인까지 걸린 일수 분포 시각화
fig = px.histogram(df_hibernated, x='days_to_last_login',
                   title="'HIBERNATED' 상태의 유저들의 마지막 로그인까지 걸린 일수 분포",
                   labels={'days_to_last_login': 'Days to Last Login'},
                   nbins=50, # Bin의 수를 조정하여 분포의 세부적인 모양을 변경할 수 있습니다.
                   color_discrete_sequence=['#636EFA']) # 색상 설정

fig.update_layout(xaxis_title="Days to Last Login",
                  yaxis_title="Count of Users",
                  bargap=0.05) # 바 간격 설정

fig.show()

## 2.2 Trend Analysis
각 Dataset(order, course, user, customer and refund)에서
2022년 1월 부터 12월까지의 데이터를 월별로 집계하여 볼 수 있는 트렌드를 하나씩만 보여주세요. Trend를 이용할 때는 어떤 차트를 이용하는 것이 좋을까요?

#### order 데이터셋 - 월별 주문 상태(state) 분포

In [ ]:
## order 데이터셋 - 월별 주문 상태(state) 분포
# 2022년 데이터 필터링 및 월별 주문 상태 집계
df_order['month'] = df_order['created_at'].dt.strftime('%Y-%m')
df_order_2022 = df_order[df_order['created_at'].dt.year == 2022]
monthly_order_status = df_order_2022.groupby(['month', 'state']).size().reset_index(name='count')

# 시각화
fig = px.bar(monthly_order_status, x='month', y='count', color='state',
             title='월별 주문상태 분포 추이 in 2022',
             labels={'count':'Number of Orders', 'month':'Month', 'state':'Order State'})
fig.show()

#### course 데이터셋 - 월별 새로운 코스 개설 트렌드

In [ ]:
## course 데이터셋 - 월별 새로운 코스 개설 트렌드

df_course['month'] = df_course['created_at'].dt.strftime('%Y-%m')
df_course_2022 = df_course[df_course['created_at'].dt.year == 2022]
monthly_course_creation = df_course_2022.groupby('month').size().reset_index(name='count')

fig = px.line(monthly_course_creation, x='month', y='count',
              title='월별 신규 코스 현황 in 2022',
              labels={'count':'Number of New Courses', 'month':'Month'})
fig.show()

#### user 데이터셋 - 월별 신규 사용자 가입 트렌드

In [ ]:
## user 데이터셋 - 월별 신규 사용자 가입 트렌드

df_user['month'] = df_user['created_at'].dt.strftime('%Y-%m')
df_user_2022 = df_user[df_user['created_at'].dt.year == 2022]
monthly_new_users = df_user_2022.groupby('month').size().reset_index(name='count')

fig = px.bar(monthly_new_users, x='month', y='count',
             title='월별 신규 가입자 수 in 2022',
             labels={'count':'Number of New Users', 'month':'Month'})
fig.show()

In [ ]:
df_customer['created_at'].unique()

array([datetime.datetime(22, 2, 4, 13, 35, tzinfo=<UTC>),
       datetime.datetime(21, 4, 20, 12, 27, tzinfo=<UTC>),
       datetime.datetime(22, 3, 21, 12, 37, tzinfo=<UTC>), ...,
       datetime.datetime(22, 6, 9, 4, 4, tzinfo=<UTC>),
       datetime.datetime(22, 3, 12, 15, 36, tzinfo=<UTC>),
       datetime.datetime(19, 10, 14, 16, 26, tzinfo=<UTC>)], dtype=object)

In [ ]:
df_customer.head()

,id,type,state,created_at,updated_at,user_id,name,phone,email
0,795202,USER,NORMAL,0022-02-04 13:35:00+00:00,0022-06-03 08:00:00+00:00,785170,USER_795202,1012345678,USER_795202@notexistemail.com
1,502884,USER,NORMAL,0021-04-20 12:27:00+00:00,0022-12-02 09:04:00+00:00,398242,USER_502884,1012345678,USER_502884@notexistemail.com
2,834104,USER,NORMAL,0022-03-21 12:37:00+00:00,0023-03-03 09:36:00+00:00,824123,USER_834104,1012345678,USER_834104@notexistemail.com
3,773199,USER,NORMAL,0022-01-14 08:04:00+00:00,0022-06-13 02:40:00+00:00,763128,USER_773199,1012345678,USER_773199@notexistemail.com
4,14932,USER,NORMAL,0019-09-01 07:48:00+00:00,0023-06-12 04:48:00+00:00,14567,USER_14932,1012345678,USER_14932@notexistemail.com


#### customer 데이터셋 - 월별 신규 고객 가입 트렌드

In [ ]:
## customer 데이터셋 - 월별 신규 고객 가입 트렌드

# created_at 및 updated_at 열의 데이터 타입 확인
print(df_customer['created_at'].dtype)

# 문자열 타입이 아니라면 문자열로 변환
df_customer['created_at'] = df_customer['created_at'].astype(str)
df_customer['updated_at'] = df_customer['updated_at'].astype(str)

# 연도의 첫 두 자리를 '20'으로 변경
df_customer['created_at'] = pd.to_datetime(df_customer['created_at'].str.replace(r'^00', '20', regex=True), errors='coerce', utc=True)
df_customer['updated_at'] = pd.to_datetime(df_customer['updated_at'].str.replace(r'^00', '20', regex=True), errors='coerce', utc=True)

# 'month' 열 생성
df_customer['month'] = df_customer['created_at'].dt.strftime('%Y-%m')

# 2022년 데이터 필터링 및 월별 신규 고객 가입 집계
df_customer_2022 = df_customer[df_customer['created_at'].dt.year == 2022]
monthly_new_customers = df_customer_2022.groupby('month').size().reset_index(name='count')

# 시각화
if not monthly_new_customers.empty:
    fig = px.bar(monthly_new_customers, x='month', y='count',
                 title='월별 신규 구매자수 현황 in 2022',
                 labels={'count': 'Number of New Customers', 'month': 'Month'},
                 color='count',
                 color_continuous_scale=px.colors.sequential.Viridis)
    fig.show()
else:
    print("No new customer registrations found for 2022.")


object


#### refund 데이터셋 - 월별 환불액 트렌드

In [ ]:
## refund 데이터셋 - 월별 환불액 트렌드

df_refund['month'] = df_refund['created_at'].dt.strftime('%Y-%m')
df_refund_2022 = df_refund[(df_refund['created_at'].dt.year == 2022) & (df_refund['state'] == 'COMPLETED')]
monthly_refund_amount = df_refund_2022.groupby('month')['amount'].sum().reset_index()

fig = px.area(monthly_refund_amount, x='month', y='amount',
              title='월별 환불 금액 in 2022',
              labels={'amount':'Refund Amount', 'month':'Month'})
fig.show()

##  추가 EDA

#### 월별 매출액

In [ ]:
## 월별 매출액 추이

# 필터링: 'COMPLETED' 상태
df_filtered = df_join[(df_join['order_state'] == 'COMPLETED')]

# 변환: 연도와 월을 추출하여 새로운 열 추가 (문자열로 변환)
df_filtered['year_month'] = df_filtered['order_created_at'].dt.to_period('M').astype(str)

# 그룹화 및 집계: 연도와 월별로 그룹화하고, 월별 매출액 집계
monthly_sales = df_filtered.groupby('year_month').agg(total_sales=('list_price', 'sum')).reset_index()

# 시각화: 월별 매출액 추이 시각화
# 눈금 값 설정: 3개월 간격으로 년-월 선택
tick_vals = monthly_sales['year_month'][::3]  # 3개월 간격으로 인덱싱

# 시각화
fig = px.line(monthly_sales, x='year_month', y='total_sales', title='월별 매출액 추이 (3개월 단위 년-월 표기)',
              labels={'year_month': '년-월', 'total_sales': '매출액'})
fig.update_xaxes(tickvals=tick_vals, tickangle=45)  # 3개월 간격으로 눈금 설정 및 라벨 각도 조정
fig.update_layout(xaxis_title='년-월', yaxis_title='매출액')
fig.show()

<ipython-input-46-997d26bb1cef>:7: UserWarning:

Converting to PeriodArray/Index representation will drop timezone information.

<ipython-input-46-997d26bb1cef>:7: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy



#### 월별 인기 강의 Top3

| year_month | order_name       | sales_count | average_list_price | average_sale_price |
|------------|------------------|-------------|--------------------|--------------------|
| 2022-01    | The RED :        | 1869        | 31,209.20          | 28,999.46          |
| 2022-01    | 초격차 패키지 :   | 1581        | 31,631.88          | 25,142.31          |
| 2022-01    | 올인원 패키지 :   | 1292        | 27,654.80          | 22,043.34          |
| 2022-02    | [평생수강 강의]   | 7301        | 26,277.22          | 21,481.99          |
| 2022-02    | The RED :        | 1267        | 28,942.38          | 28,342.54          |
| 2022-02    | 초격차 패키지 :   | 1035        | 31,333.33          | 23,072.46          |
| 2022-03    | 초격차 패키지 :   | 4504        | 27,708.70          | 24,007.55          |
| 2022-03    | 올인원 패키지 :   | 3150        | 25,469.84          | 21,561.90          |
| 2022-03    | The RED :        | 1220        | 32,991.80          | 30,303.28          |
| 2022-04    | The RED :        | 1098        | 33,078.32          | 32,076.50          |
| 2022-04    | 올인원 패키지 :   | 905         | 27,646.41          | 22,464.09          |
| 2022-04    | [12개월 구독제]   | 733         | 100,000.00         | 91,105.05          |
| 2022-05    | The RED :        | 851         | 30,599.29          | 27,685.08          |
| 2022-05    | 초격차 패키지 :   | 836         | 31,686.60          | 25,071.77          |
| 2022-05    | 올인원 패키지 :   | 682         | 25,791.79          | 20,043.99          |
| 2022-06    | 초격차 패키지 :   | 1698        | 31,160.19          | 29,087.16          |
| 2022-06    | 올인원 패키지 :   | 1319        | 26,141.02          | 23,525.40          |
| 2022-06    | The RED :        | 1098        | 28,324.23          | 27,240.44          |
| 2022-07    | 초격차 패키지 :   | 2169        | 30,276.63          | 25,984.32          |
| 2022-07    | 올인원 패키지 :   | 1678        | 24,308.70          | 21,090.58          |
| 2022-07    | The RED :        | 1108        | 26,362.82          | 25,469.31          |
| 2022-08    | The RED :        | 1378        | 25,014.51          | 23,896.95          |
| 2022-08    | Live2D로 만드     | 1199        | 20,000.00          | 19,833.19          |
| 2022-08    | 네오아카데미 : 완  | 909         | 30,000.00          | 29,229.92          |
| 2022-09    | 네오아카데미 : 완  | 1454        | 30,000.00          | 29,704.26          |


In [ ]:
# 연도와 월을 추출하여 새로운 열을 생성
df_join['year_month'] = df_join['order_created_at'].dt.to_period('M').astype(str)

# 강의명과 연도-월별로 그룹화하여 판매량(주문 수), 평균 정상가와 할인가 계산
monthly_course_sales = df_join.groupby(['year_month', 'order_name']).agg(
    sales_count=pd.NamedAgg(column='order_name', aggfunc='size'),
    average_list_price=pd.NamedAgg(column='list_price', aggfunc='mean'),
    average_sale_price=pd.NamedAgg(column='sale_price', aggfunc='mean')
).reset_index()

# 각 연도-월별로 판매량이 가장 높은 상위 3개 강의를 선별하고, 평균 정상가와 할인가도 포함
top_3_courses_per_month = monthly_course_sales.groupby('year_month').apply(
    lambda x: x.nlargest(3, 'sales_count')
).reset_index(drop=True)

# 결과 출력
print(top_3_courses_per_month)

<ipython-input-47-4e89a2bba1d2>:2: UserWarning:

Converting to PeriodArray/Index representation will drop timezone information.



   year_month  order_name  sales_count  average_list_price  average_sale_price
0     2022-01   The RED :         1869            31209.20            28999.46
1     2022-01   초격차 패키지 :         1581            31631.88            25142.31
2     2022-01   올인원 패키지 :         1292            27654.80            22043.34
3     2022-02   [평생수강 강의]         7301            26277.22            21481.99
4     2022-02   The RED :         1267            28942.38            28342.54
5     2022-02   초격차 패키지 :         1035            31333.33            23072.46
6     2022-03   초격차 패키지 :         4504            27708.70            24007.55
7     2022-03   올인원 패키지 :         3150            25469.84            21561.90
8     2022-03   The RED :         1220            32991.80            30303.28
9     2022-04   The RED :         1098            33078.32            32076.50
10    2022-04   올인원 패키지 :          905            27646.41            22464.09
11    2022-04  [12개월 구독제]          733           100

#### 월 단위 매출 집계

- 월별 총 매출, 주문 수, 고객 수, 강의 수

| order_created_at | total_sales | order_count | customer_count | course_count |
|------------------|-------------|-------------|----------------|--------------|
| 2022-01-31       | 594,730,000 | 18,387      | 14,856         | 367          |
| 2022-02-28       | 458,490,000 | 14,632      | 11,338         | 380          |
| 2022-03-31       | 403,770,000 | 12,030      | 9,785          | 359          |
| 2022-04-30       | 397,140,000 | 10,906      | 8,874          | 361          |
| 2022-05-31       | 350,240,000 | 10,783      | 8,577          | 360          |
| 2022-06-30       | 385,010,000 | 12,696      | 10,076         | 365          |
| 2022-07-31       | 467,620,000 | 14,963      | 11,720         | 364          |
| 2022-08-31       | 416,090,000 | 13,824      | 11,289         | 372          |
| 2022-09-30       | 416,300,000 | 13,086      | 10,539         | 364          |
| 2022-10-31       | 434,850,000 | 13,464      | 10,497         | 356          |
| 2022-11-30       | 473,420,000 | 12,928      | 10,037         | 358          |
| 2022-12-31       | 781,910,000 | 28,331      | 16,855         | 380          |


- 월별 주문명, 판매 수량, 평균 정가, 평균 판매가

| year_month | order_name         | sales_count | average_list_price | average_sale_price | average_discount_rate |
|------------|--------------------|-------------|--------------------|--------------------|-----------------------|
| 2022-01    | The RED :          | 1869        | 31,209.20          | 28,999.46          | 6.26                  |
| 2022-01    | 초격차 패키지 :    | 1581        | 31,631.88          | 25,142.31          | 20.49                 |
| 2022-01    | 올인원 패키지 :    | 1292        | 27,654.80          | 22,043.34          | 20.03                 |
| 2022-02    | [평생수강 강의]    | 7301        | 26,277.22          | 21,481.99          | 18.33                 |
| 2022-02    | The RED :          | 1267        | 28,942.38          | 28,342.54          | 1.73                  |
| 2022-02    | 초격차 패키지 :    | 1035        | 31,333.33          | 23,072.46          | 25.10                 |
| 2022-03    | 초격차 패키지 :    | 4504        | 27,708.70          | 24,007.55          | 12.78                 |
| 2022-03    | 올인원 패키지 :    | 3150        | 25,469.84          | 21,561.90          | 14.47                 |
| 2022-03    | The RED :          | 1220        | 32,991.80          | 30,303.28          | 7.19                  |
| 2022-04    | The RED :          | 1098        | 33,078.32          | 32,076.50          | 3.10                  |
| 2022-04    | 올인원 패키지 :    | 905         | 27,


In [ ]:
## 월 단위 매출 집계

# 'COMPLETED' 상태의 주문을 필터링
df_completed = df_join[df_join['order_state'] == 'COMPLETED']

# 시간대 정보를 제거
df_completed['order_created_at'] = df_completed['order_created_at'].dt.tz_localize(None)

# 'order_created_at'를 기반으로 월 단위(M)로 데이터를 재그룹핑하고 집계
df_monthly = df_completed.groupby(pd.Grouper(key='order_created_at', freq='M')).agg(
    total_sales=('list_price', 'sum'),
    order_count=('order_id', 'nunique'),
    customer_count=('customer_name', 'nunique'),
    course_count=('course_title', 'nunique')
).reset_index()

# 결과 확인
df_monthly

<ipython-input-48-f3b039c3f6f0>:7: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy



,order_created_at,total_sales,order_count,customer_count,course_count
0,2022-01-31,594730000,18387,14856,367
1,2022-02-28,458490000,14632,11338,380
2,2022-03-31,403770000,12030,9785,359
3,2022-04-30,397140000,10906,8874,361
4,2022-05-31,350240000,10783,8577,360
5,2022-06-30,385010000,12696,10076,365
6,2022-07-31,467620000,14963,11720,364
7,2022-08-31,416090000,13824,11289,372
8,2022-09-30,416300000,13086,10539,364
9,2022-10-31,434850000,13464,10497,356


In [ ]:
# 할인율 계산하여 새로운 열 추가
df_join['discount_rate'] = ((df_join['list_price'] - df_join['sale_price']) / df_join['list_price']) * 100

# 연도와 월을 추출하여 새로운 열을 생성
df_join['year_month'] = df_join['order_created_at'].dt.to_period('M').astype(str)

# 강의명과 연도-월별로 그룹화하여 판매량(주문 수), 평균 정상가, 할인가, 할인율 계산
monthly_course_sales = df_join.groupby(['year_month', 'order_name']).agg(
    sales_count=pd.NamedAgg(column='order_name', aggfunc='size'),
    average_list_price=pd.NamedAgg(column='list_price', aggfunc='mean'),
    average_sale_price=pd.NamedAgg(column='sale_price', aggfunc='mean'),
    average_discount_rate=pd.NamedAgg(column='discount_rate', aggfunc='mean')
).reset_index()

# 각 연도-월별로 판매량이 가장 높은 상위 3개 강의를 선별하고, 평균 정상가, 할인가, 할인율도 포함
top_3_courses_per_month = monthly_course_sales.groupby('year_month').apply(
    lambda x: x.nlargest(3, 'sales_count')
).reset_index(drop=True)

# 결과 출력
print(top_3_courses_per_month)

<ipython-input-49-87dbeb768707>:5: UserWarning:

Converting to PeriodArray/Index representation will drop timezone information.



   year_month  order_name  sales_count  average_list_price  \
0     2022-01   The RED :         1869            31209.20   
1     2022-01   초격차 패키지 :         1581            31631.88   
2     2022-01   올인원 패키지 :         1292            27654.80   
3     2022-02   [평생수강 강의]         7301            26277.22   
4     2022-02   The RED :         1267            28942.38   
5     2022-02   초격차 패키지 :         1035            31333.33   
6     2022-03   초격차 패키지 :         4504            27708.70   
7     2022-03   올인원 패키지 :         3150            25469.84   
8     2022-03   The RED :         1220            32991.80   
9     2022-04   The RED :         1098            33078.32   
10    2022-04   올인원 패키지 :          905            27646.41   
11    2022-04  [12개월 구독제]          733           100000.00   
12    2022-05   The RED :          851            30599.29   
13    2022-05   초격차 패키지 :          836            31686.60   
14    2022-05   올인원 패키지 :          682            25791.79   
15    20

In [ ]:
import plotly.graph_objects as go
from pandas.tseries.offsets import MonthEnd

# 첫 번째 그래프: 매출(라인)과 강의 수(바)
fig1 = go.Figure()

# 강의 수 바 차트 추가
fig1.add_trace(go.Bar(x=df_monthly['order_created_at'], y=df_monthly['total_sales'],
                      name='매출액', yaxis='y1'))

# 신청자 수 라인 차트 추가
fig1.add_trace(go.Scatter(x=df_monthly['order_created_at'], y=df_monthly['course_count'],
                          mode='lines+markers', name='강의 수', yaxis='y2'))

# 레이아웃 설정
fig1.update_layout(
    title='매출액과 강의 수',
    yaxis=dict(title='매출액'),
    yaxis2=dict(title='강의 수', overlaying='y', side='right'),
)

fig1.show()

In [ ]:
# 두 번째 그래프: 강의 수(바)와 신청자 수(라인)
fig2 = go.Figure()

# 강의 수 바 차트 추가
fig2.add_trace(go.Bar(x=df_monthly['order_created_at'], y=df_monthly['course_count'],
                      name='강의 수', yaxis='y1'))

# 신청자 수 라인 차트 추가
fig2.add_trace(go.Scatter(x=df_monthly['order_created_at'], y=df_monthly['customer_count'],
                          mode='lines+markers', name='신청자 수', yaxis='y2'))

# 레이아웃 설정
fig2.update_layout(
    title='강의 수와 신청자 수',
    yaxis=dict(title='강의 수'),
    yaxis2=dict(title='신청자 수', overlaying='y', side='right'),
)

fig2.show()

# 3. Data Documentation & Reporting
위의 2번 Analysis를 Documentation으로 정리하고 Data Analytics Report를 만들어 보려고 합니다.

주졔:
```
사업 현황과 실적 개선을 위한 강의 기획에 관하여   
```
문졔:
```
연말연초외 실적 부진   
```
<img width="832" alt="월별 매출 추이" src="https://github.com/jayz0417/data-analysis-portfolio/assets/80396016/ce42aa01-ce3c-44af-84ac-00cae8266a84">


해결:
```
월 매출, 유저 행태, 인기 강의를 분석한다
```
1. 강의가 부족한 것은 아닐까?
- 강의가 많았음에도, 매출이 부족한 경우가 많음
<img width="824" alt="월별 매출액과 강의수" src="https://github.com/jayz0417/data-analysis-portfolio/assets/80396016/eaf21363-4b93-459e-8804-dee271f87ae7">

2. 수강생이 부족한 것은 아닐까?
- 등록된 강의 수에 비해 강의를 신청한 고객이 적음
<img width="825" alt="월별 강의수와 신청자수" src="https://github.com/jayz0417/data-analysis-portfolio/assets/80396016/40d6a95a-f520-4638-88cf-c6606cb11332">

- 월별 신규 구매자 수 추이
<img width="819" alt="월별 신규 구매자수 현황" src="https://github.com/jayz0417/data-analysis-portfolio/assets/80396016/3338faa0-8e52-4471-8b13-a9b48d866d3d">

- 마지막 접속일의 50일 이내 휴먼 고객이 많음
<img width="823" alt="HIBERNATED 유저 분포" src="https://github.com/jayz0417/data-analysis-portfolio/assets/80396016/f9e0adca-bfd1-4538-b55e-3bff7fd1feb3">

3. 객단가가 낮아서 그런 것은 아닐까?
- 상위 20개 도수분포표

| List Price Range   | Order Count |
|---------------------|-------------|
| (0, 50,000]        | 315,251     |
| (50,000, 100,000]  | 7,848       |
| (100,000, 150,000] | 752         |
| (150,000, 200,000] | 36          |
| (200,000, 250,000] | 177         |
| (250,000, 300,000] | 54          |
| (300,000, 350,000] | 248         |
| (350,000, 400,000] | 7           |
| (400,000, 450,000] | 6           |
| (450,000, 500,000] | 4           |
| (500,000, 550,000] | 3           |
| (550,000, 600,000] | 4           |
| (600,000, 650,000] | 0           |
| (650,000, 700,000] | 0           |
| (700,000, 750,000] | 1           |
| (750,000, 800,000] | 0           |
| (800,000, 850,000] | 4           |
| (850,000, 900,000] | 2           |
| (900,000, 950,000] | 0           |
| (950,000, 1,000,000]| 0          |


- 고객당 구매 강의와 금액 분포도
<img width="677" alt="고객별 신청 강의 수와 금액 분포" src="https://github.com/jayz0417/data-analysis-portfolio/assets/80396016/76488563-4b40-439b-a64d-46f81610d1a5">

4. 주문완료하지 않은 고객이 많은 건 아닐까?
- 주문 상태별 월별 추이
<img width="829" alt="월별 주문상태 분포 추이" src="https://github.com/jayz0417/data-analysis-portfolio/assets/80396016/b18d2b9d-a5ff-4e19-94e2-207970b2b7de">

- 주문 상태별 매출액과 이용자 비율 추이
<img width="823" alt="월별 order_state에 따른 금액과 이용자비율" src="https://github.com/jayz0417/data-analysis-portfolio/assets/80396016/789fcac0-b982-4906-be70-208df3767e6f">

5. 강의 현황
- 인기있는 상위 10개 강의
<img width="824" alt="강의수와 금액 고객별 스캐터플랏" src="https://github.com/jayz0417/data-analysis-portfolio/assets/80396016/e3ff1fef-60c2-468b-a9ba-a470b0cef342">
<img width="828" alt="Top10 키워드 트리맵" src="https://github.com/jayz0417/data-analysis-portfolio/assets/80396016/660b1294-c80a-40aa-bc95-9015ea1cec3e">


6. 신규 가입자 현황
- 주중/주말 비교
<img width="828" alt="주중주말가입비중" src="https://github.com/jayz0417/data-analysis-portfolio/assets/80396016/b13aff52-7ac2-4ff6-88b2-71d71153ddc2">

- 시간대별 추이
<img width="825" alt="시간대별 가입활동" src="https://github.com/jayz0417/data-analysis-portfolio/assets/80396016/85689ace-63ae-4770-a792-7f15b0caba06">

- 팬데믹 전/후의 시간대별 추이
<img width="823" alt="시간대별가입활동 팬데믹전후" src="https://github.com/jayz0417/data-analysis-portfolio/assets/80396016/506d00fb-0375-4eee-8462-c79d7034f629">


결과:
```
1. 연말연초와 여름방학 시즌을 제외한 시기에 매출이 급감한다
2. 신규 가입자도 연밀연초, 여름방학 시즌에 증가한다
3. 인기 있는 강의는 데이터와 개발 관련 강의이다
4. 유저의 행태
- 주중 일과시간에 주로 신규 유저의 유입 많음
- 이탈 고객의 마지막 접속일은 분포상 50일이내가 많이 분포됨
- 신규 가입자는 연말연초, 여름방학 시즌 급증
- 코로나 팬데믹 전/후 변화(완화이후 급감)
```
결론:
```
1. 인기 강의와 연계할 수 있는 데이터/개발 관련 강의를 개발한다
2. 주중 일과시간에 신규 유저를 위한 프로모션을 기획한다
```

## 3.1 Data Documentation
2.1 Descriptive Analysis에서 분석한 내용들을 문서화(juptyter notebook, excel, or pdf)하여 github에 올려주세요. 분석 내용을 처음 보는 사람도 이해할 수 있게 주석과 설명을 추가하면 좋습니다.

## 3.2 Data Reporting(Dashboard)
2.1과 2.2의 답변과 수강생분들 개인의 분석을 추가하여 하나의 Dashboard를 만들어주세요. 데이터셋중 원하는 하나를 선택하여 그 하나에 대한 Dashboard를 만들어 주세요. 엑셀, 파워포인트, 태블로, 파이썬 모두 좋습니다.